In [1]:
#import libraries
import pandas as pd
import numpy as np

#import the neccessary module
from helper import Model, Auxiliary

set the data


MODEL FOR 2012 DATASET

In [2]:
#read the data
df = Model.get_csv('BPI_Challenge_2012.csv')
lengthOfDf = len(df.index)

In [3]:
#add a new column with the position of each case
df.at[0, 'position'] = 1
j = 2
for i in range(1, lengthOfDf):
    if df.at[i, 'case:concept:name'] == df.at[i - 1, 'case:concept:name']:
        df.at[i, 'position'] = j
        j = j + 1
    else :
        df.at[i, 'position'] = 1
        j = 2
df.head()

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,position
0,112.0,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,1.0
1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,2.0
2,112.0,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,3.0
3,112.0,SCHEDULE,W_Completeren aanvraag,2011-09-30 22:39:38.875000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,4.0
4,NaN,START,W_Completeren aanvraag,2011-10-01 09:36:46.437000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,5.0


In [4]:
#the mode of the activities at a certain position
df_mode = df.groupby('position')['concept:name'].agg(pd.Series.mode)

for i in range(1, len(df_mode.index)):
    if df_mode.map(type).iloc[i] != str:
        df_mode.iloc[i] = df_mode.iloc[i][0]

lengthOfUniquePosition = len(df['position'].unique())

nextTime = [0 for i in range(lengthOfUniquePosition)]
nextTimeCount = [0 for i in range(len(nextTime))]

for i in range(lengthOfDf):
    position = df.at[i, 'position'].astype(int)
    if position != 1:
        nextTime[position - 1] += Model.get_time_difference_as_number(df.at[i - 1, 'time:timestamp'], df.at[i, 'time:timestamp'])
        nextTimeCount[position - 1] += 1

for i in range(1, lengthOfUniquePosition):
    nextTime[i] = nextTime[i] / nextTimeCount[i]

In [5]:
#add a new column with the predicted activity of each case and the ground truth
for i in range(lengthOfDf - 1):
    if df.at[i, 'position'] + 1 <= len(df_mode.index):
        df.at[i, 'Next activity'] = df.at[i + 1, 'concept:name']
        df.at[i, 'Predicted next activity'] = df_mode.at[df.at[i, 'position'] + 1]
        df.at[i, 'Next time'] = Model.get_time_difference_as_number(df.at[i, 'time:timestamp'], df.at[i + 1, 'time:timestamp'])
        df.at[i, 'Predicted next time'] = nextTime[df.at[i, 'position'].astype(int)]

In [6]:
df.head()

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,position,Next activity,Predicted next activity,Next time,Predicted next time
0,112.0,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,1.0,A_PARTLYSUBMITTED,A_PARTLYSUBMITTED,0.334,0.507136
1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,2.0,A_PREACCEPTED,A_PREACCEPTED,53.026,36.799552
2,112.0,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,3.0,W_Completeren aanvraag,W_Completeren aanvraag,0.969,9013.225661
3,112.0,SCHEDULE,W_Completeren aanvraag,2011-09-30 22:39:38.875000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,4.0,W_Completeren aanvraag,W_Completeren aanvraag,39427.562,10533.094423
4,NaN,START,W_Completeren aanvraag,2011-10-01 09:36:46.437000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,5.0,A_ACCEPTED,W_Completeren aanvraag,356.871,869.155161


MODEL FOR 2017 DATASET

In [7]:
#read the data
df_2017 = Model.get_csv('BPI_Challenge_2017.csv')
lengthOfDf_2017 = len(df_2017.index)

In [8]:
#add a new column with the position of each case
df_2017.at[0, 'position'] = 1
j = 2
for i in range(1, lengthOfDf_2017):
    if df_2017.at[i, 'case:concept:name'] == df_2017.at[i - 1, 'case:concept:name']:
        df_2017.at[i, 'position'] = j
        j = j + 1
    else :
        df_2017.at[i, 'position'] = 1
        j = 2
df_2017.head()

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,position
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0


In [9]:
#the mode of the activities at a certain position
df_mode_2017 = df_2017.groupby('position')['concept:name'].agg(pd.Series.mode)

for i in range(1, len(df_mode_2017.index)):
    if df_mode_2017.map(type).iloc[i] != str:
        df_mode_2017.iloc[i] = df_mode_2017.iloc[i][0]

lengthOfUniquePosition_2017 = len(df_2017['position'].unique())

nextTime_2017 = [0 for i in range(lengthOfUniquePosition_2017)]
nextTimeCount_2017 = [0 for i in range(len(nextTime_2017))]

for i in range(lengthOfDf_2017):
    position_2017 = df_2017.at[i, 'position'].astype(int)
    if position_2017 != 1:
        nextTime_2017[position_2017 - 1] += Model.get_time_difference_as_number(df_2017.at[i - 1, 'time:timestamp'], df_2017.at[i, 'time:timestamp'])
        nextTimeCount_2017[position_2017 - 1] += 1

for i in range(1, lengthOfUniquePosition_2017):
    nextTime_2017[i] = nextTime_2017[i] / nextTimeCount_2017[i]

In [10]:
#add a new column with the predicted activity of each case and the ground truth
for i in range(lengthOfDf_2017 - 1):
    if df_2017.at[i, 'position'] + 1 <= len(df_mode_2017.index):
        df_2017.at[i, 'Next activity'] = df_2017.at[i + 1, 'concept:name']
        df_2017.at[i, 'Predicted next activity'] = df_mode_2017.at[df_2017.at[i, 'position'] + 1]
        df_2017.at[i, 'Next time'] = Model.get_time_difference_as_number(df_2017.at[i, 'time:timestamp'], df_2017.at[i + 1, 'time:timestamp'])
        df_2017.at[i, 'Predicted next time'] = nextTime_2017[df_2017.at[i, 'position'].astype(int)]

In [11]:
df_2017.head()

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,...,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,position,Next activity,Predicted next activity,Next time,Predicted next time
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,...,NaN,NaN,NaN,NaN,NaN,1.0,A_Submitted,A_Submitted,0.048,0.225521
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,...,NaN,NaN,NaN,NaN,NaN,2.0,W_Handle leads,W_Handle leads,0.422,0.229884
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,...,NaN,NaN,NaN,NaN,NaN,3.0,W_Handle leads,W_Handle leads,80.618,2415.933510
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,...,NaN,NaN,NaN,NaN,NaN,4.0,W_Complete application,W_Complete application,0.011,99.708786
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,...,NaN,NaN,NaN,NaN,NaN,5.0,A_Concept,A_Concept,0.010,6117.743506
